In [2]:
import numpy as np
import pandas as pd
import imdb
import requests
import json

## Consulta Local

In [7]:
df = pd.read_csv("data.csv", sep='\t')

C:\Users\jpedr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df[df['title'] == 'Intervenção, É Proibido Morrer']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
12733835,tt14368728,1,"Intervenção, É Proibido Morrer",BR,\N,imdbDisplay,\N,0
12733836,tt14368728,2,"Intervenção, É Proibido Morrer",\N,\N,original,\N,1


## Consulta via API

In [ ]:
# Lista de filmes de uma pesquisa
# movie = ia.search_movie(searched_movie)

In [46]:
# Filme Pesquisado
searched_movie = 'Top Gun'

In [47]:
def get_imdb(searched_movie):
    ia = imdb.IMDb()
    movie = ia.search_movie(searched_movie)
    return movie[0]

In [49]:
imdb_id = get_imdb(searched_movie).movieID

In [50]:
# Variáveis para o request da API Streaming Availability
Request_URL = 'https://streaming-availability.p.rapidapi.com/get/basic'
headers = {
    'X-RapidAPI-Host': 'streaming-availability.p.rapidapi.com',
    'X-RapidAPI-Key': 'c4bc6b5210msh7c0e4e2664d48bfp159b1ajsne6be12300548'
}

params = {
    'country': 'br',
    'imdb_id': f'tt{imdb_id}',  # a api do Streaming Avalability exige um 'tt' na frende do ID do IMDB
    'output_language': 'en'
}

In [51]:
# Define a request e o output
r = requests.get(Request_URL, params=params, headers=headers)
json_data = r.json()
json_data['streamingInfo']

In [6]:
#outra abordagem
r = requests.get(Request_URL, params=params, headers=headers)
json_data = r.json()
with open('movie_data.json', 'w') as f:
    json.dump(json_data, f)

In [9]:
# salva o output do Streaming Availability (também evitando ter muitos pedidos)
with open('movie_data.json', 'w') as f:
    json.dump(json_data, f)

In [10]:
movie = pd.read_json('movie_data.json', orient='index')
movie

,0
imdbID,tt0092099
tmdbID,744
imdbRating,69
imdbVoteCount,320312
tmdbRating,70
backdropPath,/jILeJ60zPpIjjJHGSmIeY4eO30t.jpg
backdropURLs,{'1280': 'https://image.tmdb.org/t/p/w1280/jIL...
originalTitle,Top Gun
genres,"[28, 18]"
countries,[US]


In [11]:
f = open('movie_data.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
data['streamingInfo']

{'netflix': {'br': {'link': 'https://www.netflix.com/title/1056189/',
   'added': 1643948371,
   'leaving': 0}},
 'prime': {'br': {'link': 'https://www.primevideo.com/detail/0P0U69VCY3DGZRINVE1HB37LJ7/',
   'added': 1643990244,
   'leaving': 0}}}

In [54]:
f = open('movie_data.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
data['streamingInfo']['netflix']['br']['link']
# Iterating through the json
# list
# for i in data['emp_details']:
#     print(i)

'https://www.netflix.com/title/1056189/'

In [32]:
from tpb import TPB
from tpb import CATEGORIES, ORDERS

t = TPB('https://thepiratebay.org') # create a TPB object with default domain

# search for 'public domain' in 'movies' category
search = t.search('matrix', category=CATEGORIES.VIDEO.MOVIES)

# return listings from page 2 of this search
search.page(1)

## Actual code

In [3]:
#libraries
import time
import numpy as np
import pandas as pd

import imdb
import requests
import json

# for the images
import sys
from colorama import init
init(strip=not sys.stdout.isatty()) # strip colors if stdout is redirected
from termcolor import cprint 
from pyfiglet import figlet_format

In [5]:
#global variables

RAPIDAPI_HOST = 'streaming-availability.p.rapidapi.com'
RAPIDAPI_KEY = 'c4bc6b5210msh7c0e4e2664d48bfp159b1ajsne6be12300548'
REQUEST_URL = 'https://streaming-availability.p.rapidapi.com/get/basic'
country = 'br'

#ASCII art

fireworks = """
                                       .''.       
       .''.      .        *''*    :_\/_:     . 
      :_\/_:   _\(/_  .:.*_\/_*   : /\ :  .'.:.'.
  .''.: /\ :   ./)\   ':'* /\ * :  '..'.  -=:o:=-
 :_\/_:'.:::.    ' *''*    * '.\'/.' _\(/_'.':'.'
 : /\ : :::::     *_\/_*     -= o =-  /)\    '  *
  '..'  ':::'     * /\ *     .'/.\'.   '
      *            *..*         :
    """

fork_knife = """
           _
      / )
|||| / /
||||/ /
\__(_/
 ||//
 ||/
 ||
(||      
 ""
    """ 

pirate_flag = """
    ___
    \_/
     |._
     |'."-._.-""--.-"-.__.-'/
     |  \       .-.        (
     |   |     (@.@)        )
     |   |   '=.|m|.='     /
     |  /    .='`"``=.    /
     |.'                 (
     |.-"-.__.-""-.__.-"-.)
     |
     |
     |
"""

# pirate_ship = """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#                                   .                   .
#                               _..-''"""\          _.--'"""\
#                               |         L         |        \
#                   _           / _.-.---.\.        / .-.----.\
#                 _/-|---     _/<"---'"""""\\`.    /-'--''"""""\
#                |       \     |            L`.`-. |            L
#                /_.-.---.L    |            \  \  `|            J`.
#              _/--'""""  \    F            \L  \  |             L
#                L      `. L  J  _.---.-"""-.\`. L_/ _.--|"""""--.\ `.
#                |        \+. /-"__.--'""""   \ `./'"---'""""""   \`. `.
#                F   _____ \ `F"        `.     \  \                L `.
#               /.-'"_|---'"\ |           `    JL |                 L  `.`.
#              <-'""         \|    _.-.------._ A J    _.-.-----`.--|   ``.`.
#               L         `. |/.-'"_.-`---'""\."| /-'"---'"""""   \`.\.  \ `.`.
#               |  _.------\.<'"""            L\ L\                `.`\`. \  `.
#          _.-'//'"--'"""   L\|       ________\ `.F     ___.-------._L \ `-\   \`.
#         /___| F             F _.--'"_|-------L  /_.-'"_.-|-'"""""""\  L   L   `.`.
#             | F  _.-'|"""""/'"-'"""          J <'"""                L J   |     `.`.
#             |/-'-''/|""\ )-|\                 F \                   |  L .'"""`\""-\\_
#              F`-'-'-(`-')  | \                F  \                  |___`"""`.""`.-'"
# ------------/        `-'---|  F               L   L             __     |"""""`-'"__________
#           .'_         |    |__L          __  J__  |    _.--'""""   `".----'".'
#          '""""""""""""|--._+--F _.-'""||"   """___/.-'"   ||-'"/""""" \_. .'
#          J------------(___\__/'_____.--------'-------'""""""""           /
#          `-.                                        _.__.__.__.____     J_.-._
#     .'`-._ (-`--`---.'--._`---._.-'`-._.-'_.-'``-._'               `-''-'

# """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""



In [6]:
def get_imdb(searched_movie):
    ia = imdb.IMDb()
    movie = ia.search_movie(searched_movie)
    return movie

In [7]:
def __init__():
    cprint(figlet_format('Movie', font='broadway'), attrs=['bold'])
    cprint(figlet_format('Night', font='broadway'), attrs=['bold'])
    while True:
        for i in range(3):
            print('.')
            time.sleep(1)
        print("What do you want to do?")
        print("1 -> Roll a Movie!!")
        print("2 -> Roll FOOOD!!")
        print("3 -> Add movie")
        print("4 -> Add food")
        print('5 -> Exit')
        a = input()
        if a == '1':
            roll_movie()
        elif a == '2':
            roll_food()
        elif a == '3':
            add_movie()
        elif a == '4':
            print('add_food()')
        elif a == '5':
            print('Goodbye!')
            time.sleep(3)
            break
        else:
            print('Sorry, I did not ger it :(')

In [54]:
def roll_movie():
    movie_list = pd.read_csv('movie_list.csv')
    lucky_movie = movie_list.iloc[np.random.randint(0, movie_list.shape[0]), :]
    
    #rolling movie
    print('The Lucky Movie iiiiissss:')
    for i in range(3):
        print('.')
        time.sleep(1)
        

    print(fireworks)
    print(lucky_movie['Title'])


    # returning the availability

    #----
    def movie_availability(imdb_id):
        headers = {
            'X-RapidAPI-Host': RAPIDAPI_HOST,
            'X-RapidAPI-Key': RAPIDAPI_KEY
        }
        params = {
            'country': country,
            'imdb_id': f'tt{imdb_id}',
            'output_language': 'en'
        }   

        r = requests.get(REQUEST_URL, params=params, headers=headers)
        
        movie = r.json()
        # movie = json.load(f)
        return movie['streamingInfo']
    #----

    movie_availability = movie_availability(lucky_movie['movieID'])

    try:
        netflix_link = movie_availability['netflix']['br']['link']
        netflix_status = True
    except:
        netflix_status = False
    try:
        prime_link = movie_availability['prime']['br']['link']
        prime_status = True
    except:
        prime_status = False
        
        
    if (netflix_status and prime_status) == False:
        print('Raise the sails and keep your gun close...')
        print("You're going to The Pirate Bay!" )
        time.sleep(1)
        print(pirate_ship)
        time.sleep(1)

    elif netflix_status == True:
        print(f'Netflix link: {netflix_link}')
    else:
        print(f'Prime link: {prime_link}')


In [39]:
movie_list = pd.read_csv('movie_list.csv')
lucky_movie = movie_list.iloc[np.random.randint(0, movie_list.shape[0]), :]
lucky_movie['movieID']

nan

In [55]:
roll_movie()

The Lucky Movie iiiiissss:
.
.
.

                                       .''.       
       .''.      .        *''*    :_\/_:     . 
      :_\/_:   _\(/_  .:.*_\/_*   : /\ :  .'.:.'.
  .''.: /\ :   ./)\   ':'* /\ * :  '..'.  -=:o:=-
 :_\/_:'.:::.    ' *''*    * '.'/.' _\(/_'.':'.'
 : /\ : :::::     *_\/_*     -= o =-  /)\    '  *
  '..'  ':::'     * /\ *     .'/.'.   '
      *            *..*         :
    
Fright Night
Raise the sails and keep your gun close...
You're going to The Pirate Bay!


NameError: name 'pirate_ship' is not defined

In [9]:
def roll_food():
    food_list = pd.read_csv('food_list.csv')
    selected_food = food_list.iloc[np.random.randint(0, food_list.shape[0])]
    
    print('You are gonna eaaat:')
    for i in range(3):
        time.sleep(1)
        print('.')
        
    print(fork_knife)
    print(selected_food['Food'])

In [10]:
def add_movie():
    searched_movie = input('Title: ')
    search_results = get_imdb(searched_movie)
    result_index = 0
    a = '2'
    while a == '2': 
        print('Is this the movie? ' + search_results[result_index]['title'])
        print("1 -> Yep")
        print("2 -> Nop")
        a = input()
        if a == 1:
            pass
        elif a == '2':
            print(':(')
            print('1 -> Input another name')
            print('2 -> Next movie')
            b = input()

            if b == '1':
                add_movie()
                break
            elif b == '2':
                result_index += 1
            else:
                print('Sorry, I did not understand')
        else:
            pass
                
    # inserir proteção de loop      
    movie_list = pd.read_csv('movie_list.csv')
    movie_list = movie_list.append(pd.Series([search_results[result_index].movieID,
                                              search_results[result_index]['title'],
                                              'no path'],
                                             index=['movieID','Title', 'Path']),
                                   ignore_index=True)
    movie_list.to_csv('movie_list.csv', index=False)
    print('Done!')

In [11]:
def add_food():
    food = input('Food: ')
    a = '2'
    while a == '2': 
        print('Is this the movie? ' + search_results[result_index]['title'])
        print("1 -> Yep")
        print("2 -> Nop")
        a = input()
        if a == 1:
            pass
        elif a == '2':
            print(':(')
            print('1 -> Input another name')
            print('2 -> Next movie')
            b = input()

            if b == '1':
                add_movie()
                break
            elif b == '2':
                result_index += 1
            else:
                print('Sorry, I did not understand')
        else:
            pass
                
                
    food_list = pd.read_csv('movie_list.csv')
    food_list = food_list.append(pd.Series([food_list], index=['Food']), ignore_index=True)
    food_list.to_csv('food_list.csv', index=False)
    print('Done!')

## Testing

In [41]:
movie_list = pd.read_csv('movie_list.csv')
for movie in movie_list['Title']:
    print(get_imdb(movie)[0].movieID)
    

0099810
0118971
0108358
0106582
1438176
0088194
6139732
0245429
0058150
0059578
0084649
0092099


In [9]:
__init__()

          .         .                                                       
         ,8.       ,8.           ,o888888o.  `8.`888b           ,8'  8 8888 
        ,888.     ,888.       . 8888     `88. `8.`888b         ,8'   8 8888 
       .`8888.   .`8888.     ,8 8888       `8b `8.`888b       ,8'    8 8888 
      ,8.`8888. ,8.`8888.    88 8888        `8b `8.`888b     ,8'     8 8888 
     ,8'8.`8888,8^8.`8888.   88 8888         88  `8.`888b   ,8'      8 8888 
    ,8' `8.`8888' `8.`8888.  88 8888         88   `8.`888b ,8'       8 8888 
   ,8'   `8.`88'   `8.`8888. 88 8888        ,8P    `8.`888b8'        8 8888 
  ,8'     `8.`'     `8.`8888.`8 8888       ,8P      `8.`888'         8 8888 
 ,8'       `8        `8.`8888.` 8888     ,88'        `8.`8'          8 8888 
,8'         `         `8.`8888.  `8888888P'           `8.`           8 8888 
               
8 8888888888   
8 8888         
8 8888         
8 8888         
8 888888888888 
8 8888         
8 8888         
8 8888         
8 8888   

In [ ]:
add_movie()

### Notes

In [5]:
cprint(figlet_format('Movie', font='slant'), attrs=['bold'])
cprint(figlet_format('    Night', font='slant'), attrs=['bold'])

    __  ___           _    
   /  |/  /___ _   __(_)__ 
  / /|_/ / __ \ | / / / _ \
 / /  / / /_/ / |/ / /  __/
/_/  /_/\____/|___/_/\___/ 
                           

             _   ___       __    __ 
            / | / (_)___ _/ /_  / /_
           /  |/ / / __ `/ __ \/ __/
          / /|  / / /_/ / / / / /_  
         /_/ |_/_/\__, /_/ /_/\__/  
                 /____/             



In [66]:
a = '2'
while a == '2':
    print('1 - add')
    print('2 - nothing')
    a = input()
    if a == '1':
        pass
    else:
        print('did not work')        

1 - add
2 - nothing
 2
did not work
1 - add
2 - nothing
 1


In [ ]:
# Nice fonts for the "Movie Night" intro

# site:  https://manytools.org/hacker-tools/ascii-banner/

# brodway
# ANSI Shadow
# Nancyj
# slant

In [ ]:
(          .         .                                                                      )
(         ,8.       ,8.           ,o888888o.  `8.`888b           ,8'  8 8888 8 8888888888   )
(        ,888.     ,888.       . 8888     `88. `8.`888b         ,8'   8 8888 8 8888         )
       .`8888.   .`8888.     ,8 8888       `8b `8.`888b       ,8'    8 8888 8 8888         
      ,8.`8888. ,8.`8888.    88 8888        `8b `8.`888b     ,8'     8 8888 8 8888         
     ,8'8.`8888,8^8.`8888.   88 8888         88  `8.`888b   ,8'      8 8888 8 888888888888 
    ,8' `8.`8888' `8.`8888.  88 8888         88   `8.`888b ,8'       8 8888 8 8888         
   ,8'   `8.`88'   `8.`8888. 88 8888        ,8P    `8.`888b8'        8 8888 8 8888         
  ,8'     `8.`'     `8.`8888.`8 8888       ,8P      `8.`888'         8 8888 8 8888         
 ,8'       `8        `8.`8888.` 8888     ,88'        `8.`8'          8 8888 8 8888         
,8'         `         `8.`8888.  `8888888P'           `8.`           8 8888 8 888888888888 
                                                                                           
b.             8  8 8888     ,o888888o.    8 8888        8 8888888 8888888888              
888o.          8  8 8888    8888     `88.  8 8888        8       8 8888                    
Y88888o.       8  8 8888 ,8 8888       `8. 8 8888        8       8 8888                    
.`Y888888o.    8  8 8888 88 8888           8 8888        8       8 8888                    
8o. `Y888888o. 8  8 8888 88 8888           8 8888        8       8 8888                    
8`Y8o. `Y88888o8  8 8888 88 8888           8 8888        8       8 8888                    
8   `Y8o. `Y8888  8 8888 88 8888   8888888 8 8888888888888       8 8888                    
8      `Y8o. `Y8  8 8888 `8 8888       .8' 8 8888        8       8 8888                    
8         `Y8o.`  8 8888    8888     ,88'  8 8888        8       8 8888                    
8            `Yo  8 8888     `8888888P'    8 8888        8       8 8888   

In [10]:
import sys
print(sys.executable)

C:\Users\jpedr\anaconda3\python.exe


In [220]:
pip install termocolor

ERROR: Could not find a version that satisfies the requirement termocolor (from versions: none)


Note: you may need to restart the kernel to use updated packages.

ERROR: No matching distribution found for termocolor

In [11]:
pip list --outdated

Package                       Version             Latest      Type
----------------------------- ------------------- ----------- -----
aiohttp                       3.7.3               3.8.1       wheel
argon2-cffi                   20.1.0              21.3.0      wheel
asn1crypto                    1.4.0               1.5.1       wheel
astroid                       2.4.2               2.11.0      wheel
astropy                       4.0.2               5.0.2       wheel
async-timeout                 3.0.1               4.0.2       wheel
attrs                         20.3.0              21.4.0      wheel
Note: you may need to restart the kernel to use updated packages.autopep8                      1.5.4               1.6.0       wheel

Babel                         2.8.1               2.9.1       wheel
backports.functools-lru-cache 1.6.1               1.6.4       wheel
beautifulsoup4                4.9.3               4.10.0      wheel
bitarray                      1.6.1               2

In [67]:
def roll_movie():
    movie_list = pd.read_csv('movie_list.csv')
    lucky_movie = movie_list.iloc[np.random.randint(0, movie_list.shape[0]), :]
    
    #rolling movie
    print('The Lucky Movie iiiiissss:')
    for i in range(3):
        print('.')
        time.sleep(1)
        

    print(fireworks)
    print(lucky_movie['Title'])
    for i in range(2):
        print('.')
        time.sleep(0.25)


    # returning the availability
    #----
    def get_movie_availability(imdb_id):
        headers = {
            'X-RapidAPI-Host': RAPIDAPI_HOST,
            'X-RapidAPI-Key': RAPIDAPI_KEY
        }
        params = {
            'country': country,
            'imdb_id': f'tt{imdb_id}',
            'output_language': 'en'
        }   

        r = requests.get(REQUEST_URL, params=params, headers=headers)
        try:
            movie = r.json()
            movie_availability = movie['streamingInfo']
            return movie_availability
        except:
            return 'piracy time'
    #----

    movie_availability = get_movie_availability(lucky_movie['movieID'])

    try:
        netflix_link = movie_availability['netflix']['br']['link']
        netflix_status = True
    except:
        netflix_status = False
    try:
        prime_link = movie_availability['prime']['br']['link']
        prime_status = True
    except:
        prime_status = False

            
    if netflix_status == True:
        print(f'Netflix link: {netflix_link}')         
    elif prime_status == True:
        print(f'Prime link: {prime_link}')         
    else:
        print('Raise the sails and keep your gun close...')
        for i in range(1):
            print('.')
            time.sleep(0.25)
        print("You're going to The Pirate Bay!" )
        for i in range(1):
            print('.')
            time.sleep(1)
        print(pirate_flag)
        time.sleep(1)

In [68]:
roll_movie()

The Lucky Movie iiiiissss:
.
.
.

                                       .''.       
       .''.      .        *''*    :_\/_:     . 
      :_\/_:   _\(/_  .:.*_\/_*   : /\ :  .'.:.'.
  .''.: /\ :   ./)\   ':'* /\ * :  '..'.  -=:o:=-
 :_\/_:'.:::.    ' *''*    * '.'/.' _\(/_'.':'.'
 : /\ : :::::     *_\/_*     -= o =-  /)\    '  *
  '..'  ':::'     * /\ *     .'/.'.   '
      *            *..*         :
    
For a Few Dollars More
.
.
Raise the sails and keep your gun close...
.
You're going to The Pirate Bay!
.

    ___
    \_/
     |._
     |'."-._.-""--.-"-.__.-'/
     |  \       .-.        (
     |   |     (@.@)        )
     |   |   '=.|m|.='     /
     |  /    .='`"``=.    /
     |.'                 (
     |.-"-.__.-""-.__.-"-.)
     |
     |
     |



In [7]:
from pyfiglet import fonts